In [ ]:
import numpy as np
import numpy.linalg as alg


In [ ]:
class MDP:
    def __init__(self, configs):
        self.typ = configs['typ']
        self.state = configs['init_state']
        self.H = configs['H']
        self.init_state = configs['init_state']
        self.state_space = configs['state_space']
        self.action_space = configs['action_space']     
            
        if self.typ == 'river':
            self.S = len(self.state_space)
        
    def next_state(self, action):
        typ = self.typ
        ###here to specify how to transfer
        if typ == 'river':
            if self.state == 0:
                if action == 0:
                    self.state = 1
                else:
                    p = [0.1, 0.9]
                    self.state = np.random.choice([0,1], p = p)
                return
            if self.state == self.S -1:
                if action == 0:
                    self.state = self.S-2
                else:
                    p = [0.05, 0.95]
                    self.state = np.random.choice([self.S-2, self.S-1], p = p)
                return
            else:
                if action == 0:
                    self.state -=1
                else:
                    p = [0.05, 0.05, 0.9]
                    self.state = np.random.choice([self.state-1, self.state, self.state+1], p = p)
                return
                
                    
            
    def reset(self):
        self.state = self.init_state
        print('reset')
        return 
        
    
    def reward(self, state, action):
        typ = self.typ
        if typ == 'river':
            if state == 0:
                if action == 0:
                    return 0.005
            if state == self.S - 1:
                if action == 1:
                    return 1
            return 0

    def phi(self, state, action):
        ####given state and action, return a matrix in shape d times S
        ###state is 0,1,2,..., action is arbitrary 
        
        if self.typ == 'river':
            ##here we treat river as a 3-dimensional linear mixture MDP
            ###should return 3 times S
            rr = np.zeros((3, self.S))
            if action == 1:               
                if state == 0:
                    rr[1,0] = 1
                    rr[2,0] = 1
                    rr[0,1] = 1
                if state == self.S-1:
                    rr[0,-1] = 1
                    rr[1,-2] = 1
                    rr[2,-2] = 1
                if 0<state<self.S - 1:
                    rr[0,state+1] = 1
                    rr[1, state] = 1
                    rr[2, state-1] = 1
            else:
                ss = max(0,state-1)
                rr[0,ss] = 1
                rr[1,ss] = 1
                rr[2,ss] = 1
                
            return rr
        
        
        

In [ ]:
class VTR_HF:
    def __init__(self, configs):
        self.i = 0
        
        
        self.dim = configs['dim']
        self.M = configs['M']
        self.lam = configs['lam']
        self.state_space = configs['state_space']
        self.action_space = configs['action_space']
        self.H = configs['H']
        self.beta = configs['beta']   
        self.alpha = configs['alpha']
        self.gamma = configs['gamma']

        
        ###the length of the following list is of M
        self.phi_list = []
        self.theta_list = [np.zeros((self.dim, 1))]*self.M        
        self.tilde_Sigma_list = [np.identity(self.dim)]*self.M
        self.b_list = [np.zeros((self.dim, 1))]*self.M
        self.hat_Sigma_list = [np.identity(self.dim)]*self.M
        self.v_func = 0
        ###self.v_func is the next_value function in shape S times H (v_2, _3, ..., v_H+1)
        
    def Q(self, v_next, reward, phi, state, action):
        ###here v_next is a S times 1 shape vector
        phiv = phi(state, action)@v_next        
        theta = self.theta_list[0]
        ####phiv in shape d \times 1
        v_action = reward(state, action) + theta.transpose()@phiv + \
        self.beta*np.sqrt(phiv.transpose()@alg.solve(self.hat_Sigma_list[0], phiv))
        v_action = np.clip(v_action, 0,1)[0,0]       
        return v_action
        
    def update_v(self, reward, phi):
        ##get current v function (H number) from line 4 to 6 in Algorithm 1
        ###v_func is in shape S times H
        v_func = np.zeros((len(self.state_space), 1))
        for h in range(self.H, 1, -1):
            v= np.zeros((len(self.state_space),1))
            v_next = v_func[:,0:1]
            
            for state in self.state_space:
                all_action = []
                for action in self.action_space:
                    v_action = self.Q(v_next, reward, phi, state, action)
                    all_action.append(v_action)
                v[state,0] = max(all_action)
            v_func = np.concatenate((v, v_func), axis = 1)
        
        ###v_func is in shape S times H
        self.v_func = v_func
        return 
    
    def update_hat(self):
           
        self.hat_Sigma_list = self.tilde_Sigma_list.copy()     
        self.theta_list = []            
        for m in range(self.M):
            self.theta_list.append(alg.solve(self.hat_Sigma_list[m], self.b_list[m]))
        return
    
    def select_action(self, state, h, reward, phi):
        v = self.v_func[:,h:(h+1)]
        all_action = {}
        for i in range(len(self.action_space)):
            action = self.action_space[i]
            all_action[i] = self.Q(v, reward, phi, state, action)
        
        actions = []

        for i in range(len(self.action_space)):
            if all_action[i] == all_action[max(all_action)]:
                actions.append(i)
        
        action = np.random.choice(actions)
        
        return self.action_space[action]
    
    def update(self,state, action, next_state, h, phi):
        ###from line 11 to line 14
        ####line 11
        
        self.phi_list = []
        for m in range(self.M):
            vm = self.v_func[:,h:(h+1)]**(2**m)
            phiv = phi(state, action)@vm 
            self.phi_list.append(phiv)
        ####line 12
        bar_sigma = []
        for m in range(self.M-1):
            var_1 = np.clip(self.phi_list[m+1].transpose()@self.theta_list[m+1],0,1)
            var_2 =np.clip(self.phi_list[m].transpose()@self.theta_list[m],0,1)
            variance = var_1 - var_2
            
            E_1 = np.clip(2*self.beta*np.sqrt(self.phi_list[m].transpose()@alg.solve(self.hat_Sigma_list[m], self.phi_list[m])),a_min = None, a_max= 1)
            E_2 = np.clip(self.beta*np.sqrt(self.phi_list[m+1].transpose()@alg.solve(self.hat_Sigma_list[m+1], self.phi_list[m+1])),a_min = None, a_max = 1)
            E = E_1 + E_2

            var_1 = (variance + E)[0,0]
            var_2 = self.alpha**2
            var_3 = self.gamma**2*np.sqrt(self.phi_list[m].transpose()@alg.solve(self.tilde_Sigma_list[m], self.phi_list[m]))

            variance = max(var_1, var_2, var_3)
            bar_sigma.append(variance)
        
        variance = self.gamma**2*np.sqrt(self.phi_list[-1].transpose()@alg.solve(self.tilde_Sigma_list[-1], self.phi_list[-1]))
        bar_sigma.append(max(1, self.alpha**2, variance))
        ###bar_sigma is a list of length M
        
        ###line 13 and 14
        for m in range(self.M):
            self.tilde_Sigma_list[m] = self.tilde_Sigma_list[m] + self.phi_list[m]@self.phi_list[m].transpose()/bar_sigma[m]
            self.b_list[m] = self.b_list[m] + self.v_func[next_state,h]**(2**m)*self.phi_list[m]/bar_sigma[m]
       
        return
        
    
    

In [ ]:
class Qlearning:
    
####Q learning enjoys the state space [0,1,2,...] and action space [0,1,2,...]
    def __init__(self, configs):
        self.i = 0
        
        

        self.state_space = configs['state_space']
        self.action_space = configs['action_space']
        self.H = configs['H']
        self.beta = configs['Q_beta']   
        
        self.Q_list = self.H*np.ones((self.H, len(self.state_space), len(self.action_space)))
        self.V_list = np.zeros((self.H+1, len(self.state_space)))
        ####self.num is the count in shape H times S times A
        self.num = np.zeros((self.H, len(self.state_space), len(self.action_space)))
        ###self.v_func is the next_value function in shape S times H (v_2, _3, ..., v_H+1)

    
    def select_action(self, state, h):
        q = self.Q_list[h, state, :]
        
        actions = []

        for i in range(len(q)):
            if q[i] == max(q):
                actions.append(i)
        
        action = np.random.choice(actions)        
        
        return action
    
    def update(self,state, action, next_state, h, reward):
        self.num[h, state, action] += 1
        beta = self.beta/np.sqrt(self.num[h, state, action])
        alpha = (self.H+1)/(self.H + self.num[h, state, action])
        ####update Q
        self.Q_list[h, state, action] = (1-alpha)*self.Q_list[h, state, action] + alpha*(reward(state,action) + \
                                        self.V_list[h+1, next_state] + beta)
        self.V_list[h, state] = min(self.H, max(self.Q_list[h, state, :]))
        

        return
        
    
    

In [ ]:
CONFIGS = {}

In [ ]:
#######################################

configs = {}
configs['typ'] = 'river'

#########this is the mdp part
configs['state_space'] = [0, 1,2,3,4,5]
configs['dim'] = 3
configs['action_space'] = [0,1]

configs['init_state'] = 0
configs['H'] = 500

#####this is the vtrhf part####
configs['lam'] = 0.001
configs['beta'] = 1
configs['alpha'] = 0.01
configs['gamma'] = 0.5
configs['M'] = 4

######################################
#####this is the Q learning part#############
configs['Q_beta'] = 0.05

CONFIGS['river'] = configs

In [ ]:
######### HF-UCRL-VTR+ with M = 4 ##############
para_set = [(0.001, 1, 0.01, 0.5, 4)]

K = 500
typ = 'river'
for para in para_set:
    (CONFIGS['river']['lam'], CONFIGS['river']['beta'], CONFIGS['river']['alpha'], CONFIGS['river']['gamma'], CONFIGS['river']['M']) = para
    for ii in range(10):  
        mdp = MDP(CONFIGS[typ])
        vtr_hf = VTR_HF(CONFIGS[typ])

        result = [0]
        for k in range(K):
            mdp.reset()

            ####first get value function estimate
            vtr_hf.update_v(mdp.reward, mdp.phi)
            print('episode:', k)
            ###v is of S times H shape
            rr = 0
            visited = 0
            for h in range(mdp.H):        
                if mdp.state == 5:
                    visited += 1
                state = mdp.state
                action = vtr_hf.select_action(mdp.state, h, mdp.reward, mdp.phi)
                rr += mdp.reward(state, action)
                mdp.next_state(action)
                next_state = mdp.state
                vtr_hf.update(state, action, next_state, h, mdp.phi)
            vtr_hf.update_hat()
            result.append((rr + result[-1]*(len(result)-1))/len(result))
            print(result[-1])

        file = './'+ typ+'/full_'+str(CONFIGS[typ]['lam']) + '_'+str(CONFIGS[typ]['beta'])\
        + '_' + str(CONFIGS[typ]['alpha']) +'_'+ str(CONFIGS[typ]['gamma']) + '_' + str(ii)
        np.savetxt(file, np.array(result))

In [ ]:
#########'HF-UCRL-VTR+ with M = 2'#############
para_set = [(0.001, 1, 0.01, 0.5, 2)]

K = 500
typ = 'river'
for para in para_set:
    (CONFIGS['river']['lam'], CONFIGS['river']['beta'], CONFIGS['river']['alpha'], CONFIGS['river']['gamma'], CONFIGS['river']['M']) = para
    for ii in range(10):  
        mdp = MDP(CONFIGS[typ])
        vtr_hf = VTR_HF(CONFIGS[typ])

        result = [0]
        for k in range(K):
            mdp.reset()

            ####first get value function estimate
            vtr_hf.update_v(mdp.reward, mdp.phi)
            print('episode:', k)
            ###v is of S times H shape
            rr = 0
            visited = 0
            for h in range(mdp.H):        
                if mdp.state == 5:
                    visited += 1
                state = mdp.state
                action = vtr_hf.select_action(mdp.state, h, mdp.reward, mdp.phi)
                rr += mdp.reward(state, action)
                mdp.next_state(action)
                next_state = mdp.state
                vtr_hf.update(state, action, next_state, h, mdp.phi)
            vtr_hf.update_hat()
            result.append((rr + result[-1]*(len(result)-1))/len(result))
            print(result[-1])

        file = './'+ typ+'/singlem_'+str(CONFIGS[typ]['lam']) + '_'+str(CONFIGS[typ]['beta'])\
        + '_' + str(CONFIGS[typ]['alpha']) +'_'+ str(CONFIGS[typ]['gamma']) + '_' + str(ii)
        np.savetxt(file, np.array(result))

In [ ]:
#########'UCRL-VTR+'############
para_set = [(0.001, 1, 0.01, 0, 2)]

K = 500
typ = 'river'
for para in para_set:
    (CONFIGS['river']['lam'], CONFIGS['river']['beta'], CONFIGS['river']['alpha'], CONFIGS['river']['gamma'], CONFIGS['river']['M']) = para
    for ii in range(10):  
        mdp = MDP(CONFIGS[typ])
        vtr_hf = VTR_HF(CONFIGS[typ])

        result = [0]
        for k in range(K):
            mdp.reset()

            ####first get value function estimate
            vtr_hf.update_v(mdp.reward, mdp.phi)
            print('episode:', k)
            ###v is of S times H shape
            rr = 0
            visited = 0
            for h in range(mdp.H):        
                if mdp.state == 5:
                    visited += 1
                state = mdp.state
                action = vtr_hf.select_action(mdp.state, h, mdp.reward, mdp.phi)
                rr += mdp.reward(state, action)
                mdp.next_state(action)
                next_state = mdp.state
                vtr_hf.update(state, action, next_state, h, mdp.phi)
            vtr_hf.update_hat()
            result.append((rr + result[-1]*(len(result)-1))/len(result))
            print(result[-1])

        file = './'+ typ+'/nouncern_'+str(CONFIGS[typ]['lam']) + '_'+str(CONFIGS[typ]['beta'])\
        + '_' + str(CONFIGS[typ]['alpha']) +'_'+ str(CONFIGS[typ]['gamma']) + '_' + str(ii)
        np.savetxt(file, np.array(result))

In [ ]:
#########'UCRL-VTR'############
para_set = [(0.001, 1, 0.01, 0, 1)]

K = 500
typ = 'river'
for para in para_set:
    (CONFIGS['river']['lam'], CONFIGS['river']['beta'], CONFIGS['river']['alpha'], CONFIGS['river']['gamma'], CONFIGS['river']['M']) = para
    for ii in range(10):  
        mdp = MDP(CONFIGS[typ])
        vtr_hf = VTR_HF(CONFIGS[typ])

        result = [0]
        for k in range(K):
            mdp.reset()

            ####first get value function estimate
            vtr_hf.update_v(mdp.reward, mdp.phi)
            print('episode:', k)
            ###v is of S times H shape
            rr = 0
            visited = 0
            for h in range(mdp.H):        
                if mdp.state == 5:
                    visited += 1
                state = mdp.state
                action = vtr_hf.select_action(mdp.state, h, mdp.reward, mdp.phi)
                rr += mdp.reward(state, action)
                mdp.next_state(action)
                next_state = mdp.state
                vtr_hf.update(state, action, next_state, h, mdp.phi)
            vtr_hf.update_hat()
            result.append((rr + result[-1]*(len(result)-1))/len(result))
            print(result[-1])

        file = './'+ typ+'/hoeffding_'+str(CONFIGS[typ]['lam']) + '_'+str(CONFIGS[typ]['beta'])\
        + '_' + str(CONFIGS[typ]['alpha']) +'_'+ str(CONFIGS[typ]['gamma']) + '_' + str(ii)
        np.savetxt(file, np.array(result))

In [ ]:
#########run Q learning      
K = 500
para_set = [0.005]
for para in para_set:
    typ = 'river'
    CONFIGS['river']['Q_beta'] = para
    for ii in range(10):  
        mdp = MDP(CONFIGS[typ])
        qlearning = Qlearning(CONFIGS[typ])

        result = [0]
        for k in range(K):
            mdp.reset()

            ####first get value function estimate
            print('episode:', k)
            ###v is of S times H shape
            rr = 0
            for h in range(mdp.H):        
                state = mdp.state
                action = qlearning.select_action(mdp.state, h)
                rr += mdp.reward(state, action)
                mdp.next_state(action)
                next_state = mdp.state
                qlearning.update(state, action, next_state, h, mdp.reward)


            result.append((rr + result[-1]*(len(result)-1))/len(result))
            print(result[-1])

        file = './' + typ +'/qlearning_' + str(para) + '_'+str(ii)
        np.savetxt(file, np.array(result))

In [ ]:
####run random

K = 500
typ = 'river'
for ii in range(100):  
    mdp = MDP(CONFIGS[typ])
    result = [0]
    for k in range(K):
        mdp.reset()

        ####first get value function estimate
        print('episode:', k)
        ###v is of S times H shape
        rr = 0
        visited = 0
        for h in range(mdp.H):        
            if mdp.state == 5:
                visited += 1
            state = mdp.state
            action = np.random.choice(mdp.action_space)
            rr += mdp.reward(state, action)
            mdp.next_state(action)
            next_state = mdp.state

        result.append((rr + result[-1]*(len(result)-1))/len(result))
        print(result[-1])

    file = './'+ typ+'/random_'+ str(ii)
    np.savetxt(file, np.array(result))